# <center>Agent DQN doszkalany ewolucyjnie</center>
<center><img src="https://media.giphy.com/media/dyde6O8yn4oRh7R1Vk/source.gif" width="460" height="280" /></center>

*<center>Wykonali</center>*
*<center>Jakub Gros, Sebastian Kucharzyk</center>*


## Cel i wstępna analiza problemu
<p style='text-align: justify;'>
Jako cel obraliśmy sobie stworzenie maszynowego odpowiednika człowieka grającego w grę 
komputerową(tzw. agenta), który ucząc się na własnych błędach, będzie osiągał w niej coraz to lepsze 
rezultaty. Do tego celu nadaje się m.in. paradygmat procesu uczenia nazywany uczeniem ze wzmocnieniem,
polegający na umieszczeniu uczącego się agenta w nieznanym mu dotąd środowisku i pozwoleniu na 
wykonywanie dowolnych interakcji z otoczeniem. Agent w trakcie eksploracji doświadcza wielu różnych 
sytuacji zwanych stanami. Znajdując się w danym stanie wybiera, którą z obecnie możliwych akcji 
wykona, za co  finalnie otrzymuje pewną nagrodę lub karę. Wszystkie informacje na temat ruchów i 
nagród przechowuje w pamięci, aby nastepnie na ich podstawie móc oceniać, jaka akcja da mu 
największą nagrodę.
</p>


<p style='text-align: justify;'>
Pozornie wydawałoby się, że rozwiązanie to nadaje się niemal idealnie do sytuacji 
wymagających, aby agent sam zebrał dane, na których następnie będzie się uczył, ale niestety posiada ono 
jedną znaczącą wadę - wraz z upływem czasu ilość danych w pamięci agenta rośnie, co z kolei powoduje 
spowolnienie podejmowania decyzji, a finalnie może nawet doprowadzić do zapełnienia pamięci 
fizycznej urządzenia.
</p>


<p style='text-align: justify;'>
Aby uniknąć problemów związanych z nadmiarem danych, zdecydowaliśmny się na wykorzystanie innego paradygmatu, 
który pozwala zgeneralizować zbiór danych zamiast nieskończenie go poszerzać i przeszukiwać. Dokładniej 
rzecz ujmując, wybór padł na zastosowanie głębokiego Q-learningu, czyli połączenia uczenia 
ze wzmocnieniem z uczeniem głębokim. Polega on na zastąpieniu nieograniczonej pamięci agenta siecią neuronową 
wspomaganą relatywnie małym buforem pamiętającym N ostatnich wyborów. Ponadto po każdej akcji podjętej przez agenta
z buforu wybierany jest mały fragment danych, na których następnie sieć jest dotrenowywana.
Wykonuje się to za pomocą Q-wartości, czyli pewnej funkcji  <b>Q(S, A)</b> szacującej jak bardzo opłaca się 
podjąć akcję A będąc w stanie S. Cały powyższy proces prowadzi do stworzenia modelu Deep Q-Network(DQN).
</p>


<p style='text-align: justify;'>
Tutaj zakres projektu mógłby się zakończyć, ale nie chcieliśmy się ograniczać jedynie do głębokiego Q-learningu, 
więc postanowiliśmy znaleźć zastosowanie dla innego zagadnienia z zakresu stucznej inteligencji - algorytmów 
ewolucyjnych. Patrząc na sposób działania modelu DQN można zauważyć trzy obszary, które potencjalnie nadają się 
do ulepszenia ewolucyjnego:
</p>

1. Kształt sieci neuronowej
2. Wagi połączeń międzyneuronowych
3. Dane przechowywane w pamięci agentów

<p style='text-align: justify;'>
Po dokładnym przeanalizowaniu powyższych możliwości zdecydowaliśmy się odrzucić opcje numer 1 oraz 3. 
O wyeliminowaniu manipulacji kształtem sieci neuronowej zadecydował fakt, że wolimy skupić się na
stworzeniu jednego wariantu sieci i jego dogłębnej analizie niż na pobieżnym tworzeniu wielu modeli tylko 
po to, żeby różniły się budową. Natomiast jeśli chodzi o opcję trzecią, to jej odrzucenie wynika z 
natury DQN, której agent przechowuje w pamięci jedynie mały  fragment z całego zbioru danych, 
na którym był uczony. W takim wypadku używanie go jako podstawy do dalszego ulepszania najprawdopodobniej nie 
prowadziłoby do uzyskania wymiernych korzyści. W ten sposób stwierdziliśmy, że najciekawszym i (oby) 
najefektywniejszym podejściem będzie zastosowanie algorytmu genetycznego operującego na wagach połączeń 
międzyneuronowych.
</p>


## Etapy działania
Podsumowując przeprowadzoną powyżej analizę, zdecydowaliśmy się podzielić realizację projektu na następujące 
fazy:
1. Implementacja i gromadzenie rezultatów naiwnego agenta, który podejmuje decyzje w sposób losowy. Posłuży on 
jako wyznacznik do porównywania innych modeli.
2. Implementacja i gromadzenie rezultatów agenta wykorzystującego prostą sieć neuronową.
3. Modyfikacja implementacji agenta oraz parametrów sieci neuronowej w celu poprawy osiąganych rezultatów.
4. Wytrenowanie początkowej populacji agentów i zastosowanie algorytmu genetycznego do stworzenia osobnika z 
jak najlepiej dostosowanymi wagami międzyneuronowymi.
5. Porównanie osiągniętych rezultatów i wyciągnięcie wniosków z doświadczenia.


## Sposób przeprowadzania testów
<p style='text-align: justify;'>
Aby ocenić jakość modelu, należy najpierw go przetestować pod kątem stabilności oraz wiarygodności. Model 
staje  się stabilny w momencie, gdy wyniki osiągane w kolejnych grach przyjmują zbliżone do siebie wartości. 
Aby osiągnąć stan uznawany za  stabilny, wymagane jest trenowanie modelu na odpowiedniej (nie za małej) liczbie gier. 
Z kolei jeśli chodzi o wiarygodność, to osiągane przez dany model wyniki uznajemy za wiarygodne, kiedy nie są wypaczone 
przez losowe warunki początkowe. Aby określić wiarygodność wyników, należy wielokrotnie wytrenować model i określić 
ich tendencję, na przykład poprzez wyciągnięcie z nich średniej.


Natomiast wiarygodność danego modelu testujemy za pomocą wielokrotnego trenowania go od podstaw, ponieważ wyniki po 
tylko jednej próbie mogą być wypaczone ze względu na losowe warunki początkowe. Na przykład model z bardzo dobrymi predyspozycjami 
do danego zadania może trafić na złe warunki początkowe w trakcie trenowania, przez co wyniki przez niego osiągane 
będą co najwyżej mierne. Natomiast inny model o słabych predyspozycjach do danego zadania może trafić na niemalże perfekcyjne 
warunki początkowe, przez co jego rezultaty będą złudnie dobre, jednak przy próbie ponownego jego wytrenowania rezultaty 
te będą zgodne z jego predyspozycjami. Przez to większa liczba trenowań od podstaw daje równe szanse wariantom modeli na zaprezentowanie swoich
możliwości i wiarygodną ich ocenę.


Większa ilość treningów daje agentom równe szanse na podobne wyniki

Wiarygodny sposób testowania i porównywania wyników osiąganych przez poszczególne wersje agentów 

wypracowaliśmy dopiero podczas zaawansowanych prac nad projektem (wynika to z faktu, że musieliśmy nabyć swego 
rodzaju obycie w środowisku Cart Pole). Wtedy też zauważyliśmy, że zdecydowana większość agentów zaczyna osiągać 
stabilne rezultaty po około 200 rozegranych grach. 

Ponadto każdy z agentów na początku uczy się interakcji ze 
środowiskiem w sposób losowy, więc część agentów może mieć po prostu szczęście, co również należy wziąć pod uwagę przy 
porównywaniu poszczególnych wariantów modeli ze sobą. Tak więc zdecydowaliśmy, że wariant będziemy trenować 
dokładnie trzy razy, za każdym razem przeznaczając około 100 gier na naukę i eksplorację, a następne 100 gier 
do oceny, czyli z wyników w nich osiągniętych zostanie wyciągnięta średnia, aby na samym końcu wyciągnąć średnią 
ze średnich ze wszystkich trzech trenowań danego agenta.
</p>



## Ograniczenia sprzętowe
<p style='text-align: justify;'>
W momencie gdy zaimplementowaliśmy prostego i stosunkowo uniwersalnego agenta, który mógł być 
trenowany z wykorzystaniem większości środowisk dostępnych poprzez interfejs AI Gym, okazało się, 
że nasz sprzęt nie jest wystarczająco wydajny, aby zrealizować ten projekt w akceptowalnych ramach 
czasowych. Mówiąc konkretniej, trenowanie naszego modelu do stopnia, w którym jego wyniki są 
powtarzalne i można zacząć wyciągać z nich wnioski, w najprostszym środowisku(CartPole) zajmuje 
około 10 minut, natomiast w drugim co do złożoności LunarLander czas ten wzrasta aż do 45 minut. 
Po oszacowaniu, że do zakończenia projektu będziemy musieli trenować model w różnych konfiguracjach 
blisko 100 razy, liczba 75 godzin spędzona jednynie na trenowaniu LunarLandera byłaby zbyt duża.
</p>

## Zastosowane technologie
<p style='text-align: justify;'>
Kod projektu napisaliśmy w języku Python w wersji 3.6, z którego znakomita większość wykonuje się 
na procesorze. Karta graficzna jest odpowiedzialna za przeprowadzanie operacji czerpiących wymierne 
korzyści z wielowątkowości, a więc za trenowanie sieci neuronowej. Sama sieć oraz wszelkie operacje 
z nią związane pochodzą z wysokopoziomowego API Kerasa bazującego na bibliotece TensorFlow. Jeśli 
zaś chodzi o środowisko, w którym odbywa się trenowanie, to jest to CartPole z biblioteki AI Gym.
</p>




Celem projektu jest stworzenie i wytrenowanie agenta DQN w taki sposób, aby osiągał jak najlepsze rezultaty w grze
typu Cart Pole.



## Model losowy

In [ ]:
import random
import gym
import tensorflow as tf
a = random.randint()


In [ ]:
NUM_OF_EPISODES = 200
FRAMES_PER_EPISODE = 1000
BATCH_SIZE = 32
GAME_ID = 'CartPole-v1'
CPU_ID = '/device:CPU:0'
print(0)


### Header

### Header

#### Header

##### Header

In [ ]:
import numpy